# Open-FF data set customizer
The purpose of this notebook is to provide users with a relatively simple way to convert Open-FF's full dataframe (that is in "parquet" format) into a different format and, if desired, filter the data to a smaller subset.

While this is a "code" notebook, users do not need to know anything about coding to successfully use this tool.  

In [2]:
import pandas as pd

df_url = "https://storage.googleapis.com/open-ff-common/repos/current_repo/full_df.parquet"
df = pd.read_parquet(df_url, columns=['bgCAS','UploadKey'])